<a href="https://colab.research.google.com/github/Asritha0606/GenAIcohort_May2025/blob/main/GenAIcohort_May2025_RAGAssignment_Asritha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RAG Assignment- FAQ Chatbot

Problem Statement: Help General public understand the mission of Ayushman Bharat.

Business Requirement: Develop an AI-powered assistant chatbot to help Government to enable instant resolution to public queries.


Install Required Packages

In [ ]:
!pip install -q langchain faiss-cpu gradio sentence-transformers groq langchain-groq langchain-community text-generation

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferWindowMemory
from langchain_groq import ChatGroq
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory
import gradio as gr
import re

RAG + LLM + Memory + Guardrails + Cache

Load and split document

In [ ]:
loader = TextLoader("/content/AYUSHMAN BHARAT.txt")
docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(docs)

Create embeddings and vector store

In [ ]:
!pip install -q gpt4all

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.9 MB/s eta 0:00:00


In [ ]:
# from langchain.embeddings import GPT4AllEmbeddings

# embeddings = GPT4AllEmbeddings()


In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipython-input-7-3409896792.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
#embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
import os
from google.colab import userdata
groq_api_key = userdata.get('groq_api')
os.environ["GROQ_API_KEY"] = groq_api_key

In [ ]:
llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)

Memory for 3-turn history

In [ ]:
memory = ConversationBufferWindowMemory(k=3, return_messages=True)

/tmp/ipython-input-11-2011558429.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=3, return_messages=True)


RAG QA Chain

In [ ]:

# Step 1: Question rephrasing prompt
question_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant for answering questions about Ayushman Bharat."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("human", "Given the above conversation, generate a search query for relevant information:")
])

# Step 2: History-aware retriever (rephrases the query using memory)
retriever_with_context = create_history_aware_retriever(
    llm=llm,
    retriever=retriever,
    prompt=question_prompt
)


qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert assistant answering public queries about Ayushman Bharat."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("system", "{context}"),
    ("human", "{input}")
])


# Step 4: Document answer chain
document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=qa_prompt
)

# Step 5: Final RAG chain
qa_chain = create_retrieval_chain(
    retriever=retriever_with_context,
    combine_docs_chain=document_chain
)


Domain guardrails

In [ ]:
domain_keywords = [
    "ayushman bharat", "ayushman", "bharat",
    "health", "healthcare", "hospitals",
    "nhm", "pmjay", "hwc", "insurance"
]

def is_in_domain(query: str) -> bool:
    query = query.lower()
    return any(keyword in query for keyword in domain_keywords)


In-memory cache

In [ ]:
query_cache = {}

def chat_interface(query, chat_history):
    print("Incoming query:", query)

    # Extract history in LangChain-compatible format
    formatted_history = [(user, bot) for user, bot in chat_history[-3:]]

    print("Formatted history for RAG:", formatted_history)

    if not is_in_domain(query):
        return chat_history + [[query, "❌ Sorry, I can only answer questions about Ayushman Bharat and healthcare topics."]]

    if query in query_cache:
        return chat_history + [[query, f"✅ Cached: {query_cache[query]}"]]

    result = qa_chain.invoke({
        "input": query,
        "chat_history": formatted_history  # ✅ Manual memory
    })

    answer = result["answer"]
    query_cache[query] = answer

    return chat_history + [[query, answer]]


Launch Gradio Chat UI

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("### 🤖 Ayushman Bharat FAQ Chatbot")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Ask about Ayushman Bharat...")
    clear = gr.Button("Clear")

    state = gr.State([])

    # def user_interaction(user_message, chat_history):
    #     return chat_interface(user_message, chat_history)
    def user_interaction(user_message, chat_history):
        updated_history = chat_interface(user_message, chat_history)
        return updated_history, updated_history  # ✅ return to both chatbot and state


    msg.submit(user_interaction, [msg, state], [chatbot, state])
    msg.submit(lambda: "", None, msg)
    clear.click(lambda: [], None, chatbot)



/tmp/ipython-input-15-3112454794.py:3: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


In [ ]:
demo.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://65ab8aec8a0a72d3d0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install evaluate

In [ ]:
import evaluate
from sentence_transformers import SentenceTransformer, util

# Load evaluation models
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Example test set (use your real samples)
evaluation_data = [
    {
        "question": "What is Ayushman Bharat?",
        "reference_answer": "Ayushman Bharat is a flagship scheme by the Government of India aimed at providing universal health coverage through Health and Wellness Centres and PMJAY.",
    },
    {
        "question": "What are Health and Wellness Centres?",
        "reference_answer": "They are primary care centres under Ayushman Bharat providing comprehensive healthcare including preventive, promotive, and curative services."
    },
]

# Run through your RAG system to generate predictions
predictions = []
references = []
cosine_scores = []

for example in evaluation_data:
    question = example["question"]
    reference = example["reference_answer"]

    # Get prediction from your RAG pipeline
    result = rag_chain({
        "question": question,
        "chat_history": []  # Empty or your controlled past context
    })
    prediction = result["answer"]

    # Save for BLEU/ROUGE
    predictions.append(prediction)
    references.append(reference)

    # Compute cosine similarity
    emb_ref = embedding_model.encode(reference, convert_to_tensor=True)
    emb_pred = embedding_model.encode(prediction, convert_to_tensor=True)
    cosine_score = util.cos_sim(emb_pred, emb_ref).item()
    cosine_scores.append(cosine_score)

    print(f"Q: {question}\n🧠 Reference: {reference}\n🤖 Prediction: {prediction}\n🔗 Cosine Similarity: {cosine_score:.4f}\n{'-'*70}")

# BLEU
bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
print("🎯 BLEU Score:", bleu_score)

# ROUGE
rouge_score = rouge.compute(predictions=predictions, references=references)
print("📝 ROUGE Score:", rouge_score)

# Cosine Score Summary
average_cosine = sum(cosine_scores) / len(cosine_scores)
print(f"🔗 Avg Cosine Similarity: {average_cosine:.4f}")
